In [1]:
import os
import yaml

# CLI config

In [2]:
cli = {}
cli['config'] = './config/zs_config.yml'
cli['log_dir'] = './test_log'

if cli['log_dir'] is None:
        cli['log_dir'] = input ("Enter directory to save model and logs:")
    
if not os.path.exists(cli['log_dir']):
    os.makedirs(cli['log_dir'])
else:
    print(f"{cli['log_dir']} directory: exists")

with open(cli['config'], 'r') as f:
    args = yaml.safe_load(f)

print(args.keys())

./test_log directory: exists
dict_keys(['device', 'global_macros', 'loaders', 'model', 'trainer', 'firelight'])


In [3]:
import importlib
import operator
from typing import Callable

import torch
from packaging.version import Version
from pkg_resources import DistributionNotFound
from pytorch_lightning.utilities import _module_available

#from pl_bolts.callbacks.verification.batch_gradient import BatchGradientVerification  # type: ignore


# Ported from https://github.com/PyTorchLightning/pytorch-lightning/blob/master/pytorch_lightning/utilities/imports.py
def _compare_version(package: str, op: Callable, version: str) -> bool:
    """Compare package version with some requirements.

    >>> _compare_version("torch", operator.ge, "0.1")
    True
    """
    try:
        pkg = importlib.import_module(package)
    except (ModuleNotFoundError, DistributionNotFound):
        return False
    try:
        pkg_version = Version(pkg.__version__)
    except TypeError:
        # this is mock by sphinx, so it shall return True ro generate all summaries
        return True
    return op(pkg_version, Version(version))


_NATIVE_AMP_AVAILABLE: bool = _module_available("torch.cuda.amp") and hasattr(torch.cuda.amp, "autocast")

_TORCHVISION_AVAILABLE: bool = _module_available("torchvision")
_GYM_AVAILABLE: bool = _module_available("gym")
_SKLEARN_AVAILABLE: bool = _module_available("sklearn")
_PIL_AVAILABLE: bool = _module_available("PIL")
_OPENCV_AVAILABLE: bool = _module_available("cv2")
_WANDB_AVAILABLE: bool = _module_available("wandb")
_MATPLOTLIB_AVAILABLE: bool = _module_available("matplotlib")
_TORCHVISION_LESS_THAN_0_9_1: bool = _compare_version("torchvision", operator.lt, "0.9.1")
_PL_GREATER_EQUAL_1_4 = _compare_version("pytorch_lightning", operator.ge, "1.4.0")
_PL_GREATER_EQUAL_1_4_5 = _compare_version("pytorch_lightning", operator.ge, "1.4.5")
_TORCH_ORT_AVAILABLE = _module_available("torch_ort")
_TORCH_MAX_VERSION_SPARSEML = _compare_version("torch", operator.lt, "1.10.0")
_SPARSEML_AVAILABLE = _module_available("sparseml") and _PL_GREATER_EQUAL_1_4_5 and _TORCH_MAX_VERSION_SPARSEML

if not _PIL_AVAILABLE:  # pragma: no cover
    raise ModuleNotFoundError("PIL is required but not installed.")
if not _TORCHVISION_AVAILABLE:  # pragma: no cover
    raise ModuleNotFoundError("You want to use `torchvision` which is not installed yet.")

#__all__ = ["BatchGradientVerification"]

In [4]:
print(f"_NATIVE_AMP_AVAILABLE: {_NATIVE_AMP_AVAILABLE}")

print(f"_TORCHVISION_AVAILABLE: {_TORCHVISION_AVAILABLE}")
print(f"_GYM_AVAILABLE: {_GYM_AVAILABLE}")
print(f"_SKLEARN_AVAILABLE: {_SKLEARN_AVAILABLE}")
print(f"_PIL_AVAILABLE: {_PIL_AVAILABLE}")
print(f"_OPENCV_AVAILABLE: {_OPENCV_AVAILABLE}")
print(f"_WANDB_AVAILABLE: {_WANDB_AVAILABLE}")
print(f"_MATPLOTLIB_AVAILABLE: {_MATPLOTLIB_AVAILABLE}")
print(f"_TORCHVISION_LESS_THAN_0_9_1: {_TORCHVISION_LESS_THAN_0_9_1}")
print(f"_PL_GREATER_EQUAL_1_4: {_PL_GREATER_EQUAL_1_4}")
print(f"_PL_GREATER_EQUAL_1_4_5: {_PL_GREATER_EQUAL_1_4_5}")
print(f"_TORCH_ORT_AVAILABLE: {_TORCH_ORT_AVAILABLE}")
print(f"_TORCH_MAX_VERSION_SPARSEML: {_TORCH_MAX_VERSION_SPARSEML}")
print(f"_SPARSEML_AVAILABLE: {_SPARSEML_AVAILABLE}")

_NATIVE_AMP_AVAILABLE: True
_TORCHVISION_AVAILABLE: True
_GYM_AVAILABLE: False
_SKLEARN_AVAILABLE: False
_PIL_AVAILABLE: True
_OPENCV_AVAILABLE: False
_WANDB_AVAILABLE: False
_MATPLOTLIB_AVAILABLE: True
_TORCHVISION_LESS_THAN_0_9_1: False
_PL_GREATER_EQUAL_1_4: True
_PL_GREATER_EQUAL_1_4_5: True
_TORCH_ORT_AVAILABLE: False
_TORCH_MAX_VERSION_SPARSEML: False
_SPARSEML_AVAILABLE: False


In [5]:
import os

import numpy as np
from torch.utils.data import Dataset

#from pl_bolts.utils import _PIL_AVAILABLE
#from pl_bolts.utils.warnings import warn_missing_pkg

if _PIL_AVAILABLE:
    from PIL import Image
else:  # pragma: no cover
    print(f"Warning missing pkg: Pillow")

In [6]:
print(args['loaders'])
print(args['loaders'].keys())

{'dataset_config': {'general': {'ds_class': ['utils.dataset.phosc.PHOSCZSDataset', 'utils.dataset.phosc.ImageDataset', 'utils.dataset.phosc.PhosDataset', 'utils.dataset.phosc.PhocDataset'], 'type': 'dir', 'raw': None, 'label': None, 'unseen': None, 'transforms': None}, 'train': {'raw': '/media/windowsC/SIN_Desktop/projects/ZSL_WordSpotting/unit_test_data/train', 'label': '/media/windowsC/SIN_Desktop/projects/ZSL_WordSpotting/unit_test_data/train.csv'}, 'valid': {'raw': '/media/windowsC/SIN_Desktop/projects/ZSL_WordSpotting/unit_test_data/val', 'label': '/media/windowsC/SIN_Desktop/projects/ZSL_WordSpotting/unit_test_data/val.csv'}}, 'loader_config': {'utils.dataloader.phosc.PHOSCZSDataModule': {'seed': 1234, 'drop_last': False, 'pin_memory': True, 'shuffle': True, 'batch_size': 10, 'num_workers': 10}}}
dict_keys(['dataset_config', 'loader_config'])


In [7]:
from utils.generic.parsing import recursive_parse_settings
from utils.generic.getaccess import get_class_from_str

In [8]:
dataset_config = args['loaders']['dataset_config']
loader_config = args['loaders']['loader_config']
print(dataset_config)
print(loader_config)

{'general': {'ds_class': ['utils.dataset.phosc.PHOSCZSDataset', 'utils.dataset.phosc.ImageDataset', 'utils.dataset.phosc.PhosDataset', 'utils.dataset.phosc.PhocDataset'], 'type': 'dir', 'raw': None, 'label': None, 'unseen': None, 'transforms': None}, 'train': {'raw': '/media/windowsC/SIN_Desktop/projects/ZSL_WordSpotting/unit_test_data/train', 'label': '/media/windowsC/SIN_Desktop/projects/ZSL_WordSpotting/unit_test_data/train.csv'}, 'valid': {'raw': '/media/windowsC/SIN_Desktop/projects/ZSL_WordSpotting/unit_test_data/val', 'label': '/media/windowsC/SIN_Desktop/projects/ZSL_WordSpotting/unit_test_data/val.csv'}}
{'utils.dataloader.phosc.PHOSCZSDataModule': {'seed': 1234, 'drop_last': False, 'pin_memory': True, 'shuffle': True, 'batch_size': 10, 'num_workers': 10}}


In [9]:
loader_key, loader_config_value = next(iter(loader_config.items()))
loader_class = get_class_from_str(loader_key)
phosc_loader = loader_class(dataset_config, loader_config_value)

trainset_sz = len(phosc_loader.trainset)
validset_sz = len(phosc_loader.validset)
print(f"size(trainset): {trainset_sz}, size(validset): {validset_sz}")

size(trainset): 38, size(validset): 11


In [10]:
import tqdm, math
def get_data_from_dataset(dataset, n_epochs, steps_per_epoch, batch_size):
    for epoch_idx in range(n_epochs):
        for batch_idx in range(steps_per_epoch):
            batch_dict = {}
            for idx in range(batch_size):
                sample = next(iter(dataset))
                if len(batch_dict) ==0 :
                    for key, value in sample.items():
                        batch_dict[key] = [value]
                else:
                    for key, value in sample.items():
                        batch_dict[key] += [value]
            
            for key, value in batch_dict.items():
                batch_dict[key] = np.array(value)
                
            print(f"batch_idx: {batch_idx}, sample_batch: {batch_dict.keys()}")
            
            for key,value in batch_dict.items():
                print(f"key: {value.shape}")

get_data_from_dataset(phosc_loader.trainset, 
                      10, 
                      math.ceil(trainset_sz/loader_config_value['batch_size']), 
                      loader_config_value['batch_size'])
get_data_from_dataset(phosc_loader.validset, 
                      10, 
                      math.ceil(trainset_sz/loader_config_value['batch_size']), 
                      loader_config_value['batch_size'])

batch_idx: 0, sample_batch: dict_keys(['img', 'phos', 'phoc'])
key: (10, 50, 250, 3)
key: (10, 165)
key: (10, 604)
batch_idx: 1, sample_batch: dict_keys(['img', 'phos', 'phoc'])
key: (10, 50, 250, 3)
key: (10, 165)
key: (10, 604)
batch_idx: 2, sample_batch: dict_keys(['img', 'phos', 'phoc'])
key: (10, 50, 250, 3)
key: (10, 165)
key: (10, 604)
batch_idx: 3, sample_batch: dict_keys(['img', 'phos', 'phoc'])
key: (10, 50, 250, 3)
key: (10, 165)
key: (10, 604)
batch_idx: 0, sample_batch: dict_keys(['img', 'phos', 'phoc'])
key: (10, 50, 250, 3)
key: (10, 165)
key: (10, 604)
batch_idx: 1, sample_batch: dict_keys(['img', 'phos', 'phoc'])
key: (10, 50, 250, 3)
key: (10, 165)
key: (10, 604)
batch_idx: 2, sample_batch: dict_keys(['img', 'phos', 'phoc'])
key: (10, 50, 250, 3)
key: (10, 165)
key: (10, 604)
batch_idx: 3, sample_batch: dict_keys(['img', 'phos', 'phoc'])
key: (10, 50, 250, 3)
key: (10, 165)
key: (10, 604)
batch_idx: 0, sample_batch: dict_keys(['img', 'phos', 'phoc'])
key: (10, 50, 250